Koda Maše Popovič

In [16]:
from sage.all import *

def metricna_dimenzija_usmerjenega_grafa(graf):
    
    if not isinstance(graf, DiGraph):
        return "Napaka: Podani graf ni usmerjen graf."
    
    # vozlišča grafa
    V = graf.vertices()
    
    # Izračun razdalj med vsemi pari vozlišč
    razdalje = {u: {v: graf.distance(u, v) for v in V} for u in V}
    
    # linearni program
    lp = MixedIntegerLinearProgram(maximization=False)
    x = lp.new_variable(binary=True)  # Ustvarjanje binarnih spremenljivk za vsako vozlišče
    
    # Cilj: minimizirati vsoto vseh x[v]
    lp.set_objective(sum(x[v] for v in V))
    
    # Preverjanje, ali graf izpolnjuje pogoje
    for u in V:
        for v in V:
            if u != v:
                vozlisca = [
                    w for w in V
                    if razdalje[w][u] != razdalje[w][v] and
                    razdalje[w][u] < infinity and
                    razdalje[w][v] < infinity
                ]
                
                # Če za par u, v ne obstaja ustrezno w, ne moremo izračunati metrične dimenzije
                if not vozlisca:
                    return f"Metrične dimenzije ni mogoče določiti: za par vozlišč ({u}, {v}) ne obstaja ustrezno vozlišče."
                
                # Dodamo omejitev, če obstajajo ustrezna vozlišča w
                lp.add_constraint(sum(x[w] for w in vozlisca) >= 1)
    
    # Rešitev linearnega programa
    lp.solve()
    
    # Pridobimo rezultate
    razresljiva_mnozica = [v for v in V if lp.get_values(x[v]) == 1]
    return razresljiva_mnozica, len(razresljiva_mnozica)


def clockwise_circulant_graph(n, d):
    odmiki = list(range(1, d + 1)) 
    G = digraphs.Circulant(n, odmiki)
    #plot = G.plot(layout="circular", vertex_size=300, vertex_color="skyblue", edge_color="black", 
                  #vertex_labels=True)
    #plot.show() #zanima naju samo dimenzija, zato sva odstranili del, ki nariše graf
    return G


Ali koda vrne pravilni rezulata? Preverimo na manjših grafih

In [17]:
rez = []
for n in range (4, 7):
    for d in range(2, 6):
        g = clockwise_circulant_graph(n, d)
        md = metricna_dimenzija_usmerjenega_grafa(g)
        rez.append(md)

for r in rez:
    print(r)

([0, 1], 2)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1], 2)
([0, 2], 2)
([0, 1, 2, 3], 4)
([0, 1, 2, 3], 4)
([0, 5], 2)
([0, 1, 3], 3)
([0, 1, 3], 3)
([0, 1, 2, 4, 5], 5)


Kaj pa za večje grafe?

In [ ]:
rez = []
for n in range (100, 102):
    for d in range(20, 22):
        g = clockwise_circulant_graph(n, d)
        md = metricna_dimenzija_usmerjenega_grafa(g)
        rez.append(md)

for r in rez:
    print(r)

Ali velja trditev $dim(C_n(1, \dots, k)) = k$ za $n > 2(k-1)^2$?

In [19]:
def preveri_dimenzijo(n, k):
    #funkcija ne deluje za k > n:
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    # V primeru, da je n manjsi od funkcija f(k)
    if n <= 2*(k - 1)**2:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k - 1)^2 ni izpolnjen."}

    # Ustvari cirkulantni graf
    Cn_k = clockwise_circulant_graph(n, k)

    # Izračuna razresljivo mn in pove metricno dimenzijo
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)

    # Preveri enakost
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Različne vrednosti n za $k = 3$

In [4]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo(n, 3) for n in vrednosti_n]

#Zanka izpise vsak slovar v svoji vrstici
for r in rezultati:
    print(r)

{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 6, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 3, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 30, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 40, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 50, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 60, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 70, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 80, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 90, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 100, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}


Različne vrednosti n za $k = 4$

In [5]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 4, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 6, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 20, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 30, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 40, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 50, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 60, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 70, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 80, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 90, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 100, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}


Različne vrednosti za $k = 5$

In [6]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 5, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 7, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 20, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 30, 'k': 5, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 40, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 50, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 60, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 70, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 80, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 90, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 100, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}


Različne vrednosti n za $k = 6$

In [20]:
vrednosti_n = [4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 4, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 5, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 6, 'k': 6, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 7, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 8, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 9, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 10, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 20, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 30, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 40, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 50, 'k': 6, 'napaka': 'Pogoj n > 2(k - 1)^2 ni izpolnjen.'}
{'n': 60, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 70, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 80, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 90, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 100, 'k': 6, 'dimCn_k': 6, 'trditev_vel

Vidimo, da trditev velja.

Ali bo $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 1$?

In [23]:
def preveri_dimenzijo2(n, k):
    #funkcija ne deluje za k > n:
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    # V primeru, da je n manjsi od funkcija f(k)
    if n <= 2*(k - 1)**2 - 1:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 1 ni izpolnjen."}

    # Ustvari cirkulantni graf
    Cn_k = clockwise_circulant_graph(n, k)

    # Izračuna razresljivo mn in pove metricno dimenzijo
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)

    # Preveri enakost
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Kaj se zgodi pri $k = 3$?

In [58]:
vrednosti_n = [7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo2(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 7, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 30, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 40, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 50, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 60, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 70, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 80, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 90, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 100, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}


Pri vrednosti $k = 4$:

In [62]:
vrednosti_n = [16, 17, 18, 19, 20, 30, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo2(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 16, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 17, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 20, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 30, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 40, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 50, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 60, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 70, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 80, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 90, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 100, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}


Pri vrednosti $k = 5$

In [60]:
vrednosti_n = [31, 32, 33, 34, 40, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo2(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 31, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 32, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 33, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 34, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 40, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 50, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 60, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 70, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 80, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 90, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 100, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}


In [ ]:
vrednosti_n = [23, 24, 25, 50, 60, 70, 80, 90, 100]
rezultati = [preveri_dimenzijo2(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 42, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 43, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 44, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 45, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 50, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 60, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 70, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 80, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 90, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 100, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}


Opazimo, da je metrična dimenzija še vedno enaka $k$.

Ali bo $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 2$?

In [ ]:
def preveri_dimenzijo3(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 2:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 2 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [ ]:
vrednosti_n = [6, 7, 8, 9, 10, 20, 30, 40, 50]
rezultati = [preveri_dimenzijo3(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 3, 'k': 3, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 4, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 5, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 6, 'k': 3, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 30, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 40, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 50, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}


Za $k = 4$

In [ ]:
vrednosti_n = [ 16, 17, 18, 19, 20, 21, 22, 23, 30, 40, 50]
rezultati = [preveri_dimenzijo3(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 16, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 20, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 21, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 22, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 23, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 24, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 25, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 26, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 27, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 28, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 28, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 30, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 40, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 50, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}


Za $k = 5$

In [63]:
vrednosti_n = [30, 31, 32, 33, 34, 35, 40, 50]
rezultati = [preveri_dimenzijo3(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 30, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 31, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 32, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 33, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 34, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 35, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 40, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 50, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}


Za $k = 6$

In [68]:
vrednosti_n = [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
rezultati = [preveri_dimenzijo3(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 48, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 49, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 50, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 51, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 52, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 53, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 54, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 55, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 56, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 57, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 58, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 59, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 60, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}


Za $k = 7$

In [70]:
vrednosti_n = [70, 71, 72, 73, 74, 75]
rezultati = [preveri_dimenzijo3(n, 7) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 70, 'k': 7, 'napaka': 'Pogoj n > 2(k-1)^2 - 1 ni izpolnjen.'}
{'n': 71, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 72, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 73, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 74, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 75, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}


Tudi tu vidimo, da je metrična dimenzija grafa še vedno enaka $k$.

Kaj pa za $dim(C_n(1, \dots, k)) = k$ tudi za $n > 2(k-1)^2 - 5$? Ali sedaj pride do odstopanj?

In [48]:
def preveri_dimenzijo4(n, k):
    if k >= n:
        return {"n": n, 
                "k": k, 
                "napaka": "Za cirkulantni graf mora veljati k < n."}
    
    if n <= 2*(k - 1)**2 - 5:
        return {"n": n, 
                "k": k, 
                "napaka": "Pogoj n > 2(k-1)^2 - 5 ni izpolnjen."}

    Cn_k = clockwise_circulant_graph(n, k)
    razresljiva_mnozica, dim = metricna_dimenzija_usmerjenega_grafa(Cn_k)
    rezultat = (dim == k)

    return {"n": n, 
            "k": k, 
            "dimCn_k": dim, 
            "trditev_velja": rezultat}

Za $k = 3$

In [49]:
vrednosti_n = [3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50]
rezultati = [preveri_dimenzijo4(n, 3) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 3, 'k': 3, 'napaka': 'Za cirkulantni graf mora veljati k < n.'}
{'n': 4, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 5, 'k': 3, 'dimCn_k': 2, 'trditev_velja': False}
{'n': 6, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 7, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 8, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 9, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 10, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 20, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 30, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 40, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}
{'n': 50, 'k': 3, 'dimCn_k': 3, 'trditev_velja': True}


Za $k = 4$

In [50]:
vrednosti_n = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 40]
rezultati = [preveri_dimenzijo4(n, 4) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 8, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 9, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 10, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 11, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 12, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 13, 'k': 4, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 14, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 15, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 16, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 17, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 18, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 19, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 20, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 30, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}
{'n': 40, 'k': 4, 'dimCn_k': 4, 'trditev_velja': True}


Za $k = 5$

In [52]:
vrednosti_n = [26, 27, 28, 29, 30, 40, 50]
rezultati = [preveri_dimenzijo4(n, 5) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 26, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 27, 'k': 5, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 28, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 29, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 30, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 40, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}
{'n': 50, 'k': 5, 'dimCn_k': 5, 'trditev_velja': True}


Za $k = 6$

In [54]:
vrednosti_n = [45, 46, 47, 48, 49, 50]
rezultati = [preveri_dimenzijo4(n, 6) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 45, 'k': 6, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 46, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 47, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 48, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 49, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}
{'n': 50, 'k': 6, 'dimCn_k': 6, 'trditev_velja': True}


Za $k = 7$

In [57]:
vrednosti_n = [67, 68, 69, 70, 71, 72, 73]
rezultati = [preveri_dimenzijo4(n, 7) for n in vrednosti_n]

for r in rezultati:
    print(r)

{'n': 67, 'k': 7, 'napaka': 'Pogoj n > 2(k-1)^2 - 5 ni izpolnjen.'}
{'n': 68, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 69, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 70, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 71, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 72, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}
{'n': 73, 'k': 7, 'dimCn_k': 7, 'trditev_velja': True}


Opazimo, da metrična dimenzija grafa ne bo več enaka $k$ pri manjših $n$. Torej sva $f(k)$ tako zmanjšali,
da enakost ne velja več. Našli sva spodnjo mejo, pri kateri enakost ne velja več.

Sedaj si bova ogledali še vse $f(k)$ za katere velja: $2(k - 1)^2 - 5 < f(k) < 2(k - 1)^2$